In [ ]:
!pip install numpy==1.24.0 scikit-learn==1.3.0 pandas==2.0.3 lightautoml==0.4.1

In [ ]:
import pandas as pd
from lightautoml.automl.presets.tabular_presets import TabularAutoML
from lightautoml.tasks import Task

In [ ]:
train_df = pd.read_csv('/content/train.csv')
test_df = pd.read_csv('/content/test.csv')

In [ ]:
train_df.shape

In [ ]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
TARGET_NAME= 'target'
# Разделим данные на признаки и целевую переменную
X = train_df.drop(columns=[TARGET_NAME])
y = train_df[TARGET_NAME]

# Применим SMOTE к обучающим данным
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Создаем новый DataFrame с сбалансированными данными
train_df_resampled = pd.DataFrame(X_resampled, columns=X.columns)
train_df_resampled[TARGET_NAME] = y_resampled

# Проверим размеры классов после применения SMOTE
print(y_resampled.value_counts())



In [ ]:
# import pandas as pd
# from sklearn.preprocessing import Binarizer

# def binarize_features_by_mean(train_df):
#     # Создание копии DataFrame, чтобы не изменять исходный
#     train_df_copy = train_df.copy()

#     # Перебираем все признаки (кроме 'target')
#     for column in train_df.columns:
#         if column != 'target':  # Исключаем колонку 'target'
#             # Вычисляем среднее значение признака
#             mean_value = train_df[column].mean()

#             # Создаем объект Binarizer с порогом равным среднему значению
#             binarizer = Binarizer(threshold=mean_value)

#             # Применяем Binarizer к текущему признаку
#             train_df_copy[column + '_binarized'] = binarizer.fit_transform(train_df[[column]])

#     return train_df_copy


# # Применяем функцию
# binned_df_train = binarize_features_by_mean(train_df)

# binned_df_train.shape

In [ ]:
import pandas as pd
from sklearn.preprocessing import PolynomialFeatures

def generate_new_features(train_df):
    train_df_copy = train_df.copy()

    # Пример генерации новых признаков
    train_df_copy['perimeter_to_area'] = train_df_copy['perimeter'] / train_df_copy['area']
    train_df_copy['axis_ratio'] = train_df_copy['major_axis'] / train_df_copy['minor_axis']
    train_df_copy['area_to_convex'] = train_df_copy['area'] / (train_df_copy['major_axis'] * train_df_copy['minor_axis'])
    train_df_copy['compactness_2'] = (train_df_copy['perimeter'] ** 2) / (4 * 3.1416 * train_df_copy['area'])

        # Генерация новых признаков
    train_df_copy['area_eccentricity'] = train_df_copy['area'] * train_df_copy['eccentricity']
    train_df_copy['perimeter_eccentricity'] = train_df_copy['perimeter'] * train_df_copy['eccentricity']
    train_df_copy['roundness_solidity'] = train_df_copy['roundness'] * train_df_copy['solidity']
    train_df_copy['major_minor_axis_squared'] = (train_df_copy['major_axis'] ** 2) / (train_df_copy['minor_axis'] ** 2)
    train_df_copy['corrected_area_axis'] = train_df_copy['area'] / (train_df_copy['major_axis'] + train_df_copy['minor_axis'])
    train_df_copy['compactness_roundness'] = train_df_copy['compactness'] * train_df_copy['roundness']
    train_df_copy['eccentricity_area_perimeter'] = (train_df_copy['eccentricity'] * train_df_copy['area']) / train_df_copy['perimeter']
    # Генерация новых признаков, основанных на важнейших
    train_df_copy['minor_axis_shapefactor_1'] = train_df_copy['minor_axis'] * train_df_copy['shapefactor_1']
    # train_df_copy['minor_axis_area_to_convex'] = train_df_copy['minor_axis'] * train_df_copy['area_to_convex']
    train_df_copy['shapefactor_1_area_to_convex'] = train_df_copy['shapefactor_1'] * train_df_copy['area_to_convex']





    # # Генерация полиномиальных признаков (степень 2)
    # poly = PolynomialFeatures(degree=2, include_bias=False)
    # poly_features = poly.fit_transform(train_df_copy[['area', 'perimeter', 'major_axis', 'minor_axis']])

    # # Получаем имена для новых полиномиальных признаков
    # poly_columns = poly.get_feature_names_out(['area', 'perimeter', 'major_axis', 'minor_axis'])

    # # Создаем DataFrame для полиномиальных признаков
    # poly_df = pd.DataFrame(poly_features, columns=poly_columns)

    # # Добавляем полиномиальные признаки в исходный DataFrame
    # train_df_copy = pd.concat([train_df_copy, poly_df], axis=1)

    return train_df_copy


# Применяем функцию для генерации новых признаков
new_feature = generate_new_features(train_df_resampled)


# Применяем функцию для генерации новых признаков
new_feature_test = generate_new_features(test_df)


# Печатаем результат
new_feature.shape


In [ ]:
# import pandas as pd
# from sklearn.preprocessing import PolynomialFeatures

# def generate_polynomial_features(train_df, degree=2):
#     # Создание копии DataFrame, чтобы не изменять исходный
#     train_df_copy = train_df.copy()

#     # Перебираем все признаки (кроме 'target')
#     for column in train_df.columns:
#         if column != 'target':  # Исключаем колонку 'target'
#             # Получаем данные только для одного признака
#             data = train_df[[column]]

#             # Создаем объект PolynomialFeatures с заданной степенью
#             poly = PolynomialFeatures(degree=degree, include_bias=False)

#             # Применяем PolynomialFeatures
#             poly_features = poly.fit_transform(data)

#             # Добавляем новые полиномиальные признаки в DataFrame
#             poly_columns = [f"{column}_poly_{i}" for i in range(poly_features.shape[1])]
#             poly_df = pd.DataFrame(poly_features, columns=poly_columns)

#             # Добавляем полиномиальные признаки в исходный DataFrame
#             train_df_copy = pd.concat([train_df_copy, poly_df], axis=1)

#     return train_df_copy



# # Применяем функцию для генерации полиномиальных признаков (степень 2)
# binned_df_train = generate_polynomial_features(train_df, degree=2)

# # Печатаем результат
# binned_df_train.shape


In [ ]:
import pandas as pd
from sklearn.preprocessing import KBinsDiscretizer

def discretize_features(train_df):
    # Создание копии DataFrame, чтобы не изменять исходный
    train_df_copy = train_df.copy()

    # Перебираем все признаки (кроме 'target')
    for column in train_df.columns:
        if column != 'target':  # Исключаем колонку 'target'
            # Применяем KBinsDiscretizer
            k_bins = KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='uniform',subsample=None)  # 5 интервалов
            # Дискретизируем признак
            train_df_copy[column + '_discretized'] = k_bins.fit_transform(train_df[[column]]).astype(int)

    return train_df_copy

# Применяем функцию
binned_df_train = discretize_features(new_feature)
binned_df_test = discretize_features(new_feature_test)

binned_df_train.shape

In [ ]:
# import pandas as pd
# from sklearn.preprocessing import KBinsDiscretizer

# def discretize_features(train_df):
#     # Создание копии DataFrame, чтобы не изменять исходный
#     train_df_copy = train_df.copy()

#     # Перебираем все признаки (кроме 'target')
#     for column in train_df.columns:
#         if column != 'target':  # Исключаем колонку 'target'
#             # Применяем KBinsDiscretizer
#             k_bins = KBinsDiscretizer(n_bins=5, encode='onehot', strategy='uniform', subsample=None)  # 5 интервалов
#             # Дискретизируем признак
#             binned_data = k_bins.fit_transform(train_df[[column]])

#             # Получаем имена новых колонок для one-hot кодирования
#             onehot_columns = [f"{column}_bin_{i}" for i in range(binned_data.shape[1])]

#             # Преобразуем разреженную матрицу в плотную и добавляем в DataFrame
#             binned_df = pd.DataFrame(binned_data.toarray(), columns=onehot_columns)

#             # Добавляем новые колонки к DataFrame
#             train_df_copy = pd.concat([train_df_copy, binned_df], axis=1)

#     return train_df_copy

# # Применяем функцию
# binned_df_train = discretize_features(train_df)

# # Проверяем результат
# binned_df_train.shape


In [ ]:
# # Разделяем данные
# from sklearn.model_selection import train_test_split
# train_df, test_df_val = train_test_split(binned_df_train, test_size=0.08, random_state=42, stratify=binned_df_train['target'])

In [ ]:
train_df.shape

In [ ]:
test_df_val.shape

In [ ]:
from sklearn.metrics import roc_auc_score
import numpy as np
automl = TabularAutoML(task=Task('binary'),
                       timeout=1000,
                       cpu_limit=4,
                      reader_params={'n_folds': 5, 'random_state': 42})
# Выделяем целевую переменную 'Survived'
TARGET_NAME = 'target'
# Укажем роли переменных (добавим 'drop' для нерелевантных колонок)
roles = {'target': TARGET_NAME}  # эти колонки не было в Kaggle версии

automl.fit_predict(binned_df_train, roles=roles, verbose=1)

# Предсказание на тестовых данных
test_pred = automl.predict(binned_df_test)
test_pred

test_pred_int = (test_pred.data[:, 0]>0.5).astype(int)
results_df = pd.DataFrame({
    'target': test_pred_int
}, index=binned_df_test.index)


In [ ]:
train_df

In [ ]:
from sklearn.metrics import roc_auc_score

test_roc_auc = roc_auc_score(test_df_val['target']
                             , results_df['target'])

print(f'ROC AUC на тестовых данных: {test_roc_auc:.4f}')

In [ ]:
test_df_val['target']

In [ ]:
results_df['target']

In [ ]:

# Fast feature importances calculation
fast_fi = automl.get_feature_scores('fast')
fast_fi.set_index('Feature')['Importance'].plot.bar(figsize = (30, 10), grid = True)

In [ ]:
fast_fi

In [ ]:

train_df['target'].hist()

In [ ]:
import plotly.express as px
target = 'target'
for feature in train_df.columns:
    if feature != target:
        fig = px.histogram(train_df, x=feature, color=target,
                          nbins=30, barmode='overlay')
        fig.show()

In [ ]:
train_df

In [ ]:
train_df.columns.tolist()

In [ ]:
results_df.to_csv('answers.csv',index=False,header=False)